In [1]:
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


In [2]:
import pandas as pd
import numpy as np
from scipy.stats import pearsonr

In [3]:
train = pd.read_csv('/content/drive/MyDrive/국민대_무역_공모전/데이터/train.csv')
sub = pd.read_csv('/content/drive/MyDrive/국민대_무역_공모전/데이터/sample_submission.csv')

## Step 1: 월별 시계열 데이터 생성

In [4]:
monthly = train.groupby(['item_id', 'year', 'month']).agg({
    'value': 'sum',
    'weight': 'sum',
    'quantity': 'sum',
    'hs4': 'first'
}).reset_index()

monthly['year_month'] = pd.to_datetime(
    monthly['year'].astype(str) + '-' + monthly['month'].astype(str),
    format='%Y-%m'
)

pivot_value = monthly.pivot_table(
    index='year_month',
    columns='item_id',
    values='value',
    fill_value=0
).sort_index()

# Transpose: item을 index로 (예시 코드 형식)
pivot = pivot_value.T  # (100 items × 43 months)

pivot.head(5)

year_month,2022-01-01,2022-02-01,2022-03-01,2022-04-01,2022-05-01,2022-06-01,2022-07-01,2022-08-01,2022-09-01,2022-10-01,...,2024-10-01,2024-11-01,2024-12-01,2025-01-01,2025-02-01,2025-03-01,2025-04-01,2025-05-01,2025-06-01,2025-07-01
item_id,,,,,,,,,,,,,,,,,,,,,
AANGBULD,14276.0,52347.0,53549.0,0.0,26997.0,84489.0,0.0,0.0,0.0,0.0,...,428725.0,144248.0,26507.0,25691.0,25805.0,0.0,38441.0,0.0,441275.0,533478.0
AHMDUILJ,242705.0,120847.0,197317.0,126142.0,71730.0,149138.0,186617.0,169995.0,140547.0,89292.0,...,123085.0,143451.0,78649.0,125098.0,80404.0,157401.0,115509.0,127473.0,89479.0,101317.0
ANWUJOKX,0.0,0.0,0.0,63580.0,81670.0,26424.0,8470.0,0.0,0.0,80475.0,...,0.0,0.0,0.0,27980.0,0.0,0.0,0.0,0.0,0.0,0.0
APQGTRMF,383999.0,512813.0,217064.0,470398.0,539873.0,582317.0,759980.0,216019.0,537693.0,205326.0,...,683581.0,2147.0,0.0,25013.0,77.0,20741.0,2403.0,3543.0,32430.0,40608.0
ATLDMDBO,143097177.0,103568323.0,118403737.0,121873741.0,115024617.0,65716075.0,146216818.0,97552978.0,72341427.0,87454167.0,...,60276050.0,30160198.0,42613728.0,64451013.0,38667429.0,29354408.0,42450439.0,37136720.0,32181798.0,57090235.0


In [5]:
item_to_hs4 = monthly.groupby('item_id')['hs4'].first().to_dict()

## Step 2: HS4 매핑 및 유사도 함수

In [ ]:
# # 2-1. 품목별 HS4 매핑
# item_to_hs4 = monthly.groupby('item_id')['hs4'].first().to_dict()

# def hs4_similarity(item_a, item_b):
#     """
#     HS4 코드 앞자리 유사도 계산
#     예: 8471 vs 8473 -> 2자리 일치 -> 0.5
#     """
#     str_a = str(item_a).zfill(4)
#     str_b = str(item_b).zfill(4)

#     if str_a[:2] == str_b[:2]:  # 2자리 일치
#         return 0.5
#     elif str_a[:1] == str_b[:1]:  # 1자리 일치
#         return 0.3
#     return 0.0

## Step 3: 통계적 유의성 검증

In [6]:
from scipy.stats import pearsonr

def safe_corr_with_pvalue(x, y):
    if np.std(x) == 0 or np.std(y) == 0:
        return 0.0, 1.0
    corr, p_value = pearsonr(x, y)
    return float(corr), float(p_value) # 피어슨 상관계수, p-value

## Step 4: Granger Causality 검증

- A의 과거 정보가 B의 미래를 예측하는 데 도움이 되는가?"**를 통계적으로 검증하는 방법



```
# B의 과거 + A의 과거 사용
B(3월) = f(B의 1월, B의 2월, A의 1월, A의 2월)
B(4월) = f(B의 2월, B의 3월, A의 2월, A의 3월)
...

예측 오차 = 2  ← 더 정확!
```

**결론**
```
A의 정보를 추가했더니 예측이 더 정확해짐!
→ "A가 B를 Granger-cause 한다" ✅
```

---

🔬 통계적 검증

**가설 검정**
```
H0 (귀무가설): A의 과거 정보는 B 예측에 도움 안 됨
H1 (대립가설): A의 과거 정보는 B 예측에 도움 됨

→ F-test로 검증
→ p-value < 0.05 이면 H0 기각 → A가 B를 cause함!
```



In [7]:
from statsmodels.tsa.stattools import grangercausalitytests

def granger_test(x, y, max_lag=6):
    """
    x가 y를 Granger-cause 하는지 검증
    """
    try:
        data = pd.DataFrame({'x': x, 'y': y})
        test_result = grangercausalitytests(data[['y', 'x']],
                                            maxlag=max_lag,
                                            verbose=False)
        # 각 lag의 p-value 중 최소값 반환
        p_values = [test_result[lag][0]['ssr_ftest'][1]
                   for lag in range(1, max_lag+1)]
        return min(p_values)
    except:
        return 1.0


## Step 5: 공행성 강도 점수 계산

In [8]:
def calculate_score_advanced(corr, p_value, granger_p, lag):
    """
    정교한 공행성 점수 계산
    """

    # 1. 상관계수 점수 (0~50점) - 비선형 스케일링
    # 강한 상관관계에 더 높은 보상
    if abs(corr) >= 0.8:
        corr_score = 50
    elif abs(corr) >= 0.7:
        corr_score = 45
    elif abs(corr) >= 0.6:
        corr_score = 38
    elif abs(corr) >= 0.5:
        corr_score = 30
    else:
        corr_score = abs(corr) * 50  # 0.5 미만은 선형

    # 2. 통계적 유의성 점수 (0~25점) - 더 세분화
    if p_value < 0.001:
        sig_score = 25  # 매우 강한 유의성
    elif p_value < 0.01:
        sig_score = 20
    elif p_value < 0.05:
        sig_score = 12
    else:
        sig_score = 5  # 약한 유의성도 약간 점수

    # 3. Granger 인과성 점수 (0~20점) - 더 세분화
    if granger_p < 0.001:
        granger_score = 20  # 매우 강한 인과성
    elif granger_p < 0.01:
        granger_score = 16
    elif granger_p < 0.05:
        granger_score = 10
    elif granger_p < 0.10:
        granger_score = 5  # 약한 인과성
    else:
        granger_score = 0

    # 4. Lag 점수 (0~5점) - 비선형, 너무 짧거나 긴 lag 패널티
    # 최적 lag는 2-3개월로 가정
    if lag == 2 or lag == 3:
        lag_score = 5  # 최적
    elif lag == 1 or lag == 4:
        lag_score = 4  # 양호
    elif lag == 5:
        lag_score = 2  # 보통
    else:  # lag >= 6
        lag_score = 0  # 너무 김

    total_score = corr_score + sig_score + granger_score + lag_score

    return total_score

## Step 6: 각 Leading별 Following 탐지

In [9]:
from tqdm import tqdm

def find_comovement_pairs(
    pivot,
    max_lag=6,
    min_nonzero=12,
    corr_threshold=0.4,
    score_threshold=40
):
    items = pivot.index.to_list()
    months = pivot.columns.to_list()
    n_months = len(months)
    results = []

    for i, leader in tqdm(enumerate(items), total=len(items)):
        x = pivot.loc[leader].values.astype(float)
        if np.count_nonzero(x) < min_nonzero:
            continue

        candidates = []

        for follower in items:
            if follower == leader:
                continue

            y = pivot.loc[follower].values.astype(float)
            if np.count_nonzero(y) < min_nonzero:
                continue

            best_lag = None
            best_corr = 0.0
            best_p_value = 1.0

            # lag 탐색
            for lag in range(1, max_lag + 1):
                if n_months <= lag:
                    continue

                corr, p_value = safe_corr_with_pvalue(x[:-lag], y[lag:])

                if abs(corr) > abs(best_corr):
                    best_corr = corr
                    best_lag = lag
                    best_p_value = p_value

            # 기본 임계값 통과 시
            if best_lag is not None and abs(best_corr) >= corr_threshold:
                # Granger test
                granger_p = granger_test(x, y, max_lag=best_lag)

                # 종합 점수 계산
                score = calculate_score_advanced(
                    best_corr, best_p_value, granger_p, best_lag
                )

                # 점수 임계값 통과 시 후보에 추가
                if score >= score_threshold:
                    candidates.append({
                        "following_item_id": follower,
                        "best_lag": best_lag,
                        "max_corr": best_corr,
                        "p_value": best_p_value,
                        "granger_p_value": granger_p,
                        "comovement_score": score,
                    })

        # 알파벳 순으로 정렬
        candidates.sort(key=lambda x: x['following_item_id'])

        for candidate in candidates:
            results.append({
                "leading_item_id": leader,
                "following_item_id": candidate["following_item_id"],
                "best_lag": candidate["best_lag"],
                "max_corr": candidate["max_corr"]
            })

    pairs = pd.DataFrame(results)
    return pairs

# 실행
pairs = find_comovement_pairs(pivot)
print("탐색된 공행성쌍 수:", len(pairs))
pairs.head(20)

  0%|          | 0/100 [00:00<?, ?it/s]/usr/local/lib/python3.12/dist-packages/statsmodels/tsa/stattools.py:1556: FutureWarning: verbose is deprecated since functions should not print results
  warnings.warn(
/usr/local/lib/python3.12/dist-packages/statsmodels/tsa/stattools.py:1556: FutureWarning: verbose is deprecated since functions should not print results
  warnings.warn(
/usr/local/lib/python3.12/dist-packages/statsmodels/tsa/stattools.py:1556: FutureWarning: verbose is deprecated since functions should not print results
  warnings.warn(
/usr/local/lib/python3.12/dist-packages/statsmodels/tsa/stattools.py:1556: FutureWarning: verbose is deprecated since functions should not print results
  warnings.warn(
/usr/local/lib/python3.12/dist-packages/statsmodels/tsa/stattools.py:1556: FutureWarning: verbose is deprecated since functions should not print results
  warnings.warn(
/usr/local/lib/python3.12/dist-packages/statsmodels/tsa/stattools.py:1556: FutureWarning: verbose is deprecated

탐색된 공행성쌍 수: 1371


,leading_item_id,following_item_id,best_lag,max_corr
0,AANGBULD,APQGTRMF,5,-0.443984
1,AANGBULD,DEWLVASR,6,0.640221
2,AANGBULD,DNMPSKTB,4,-0.410635
3,AANGBULD,EVBVXETX,6,0.436623
4,AANGBULD,FTSVTTSR,3,0.531400
5,AANGBULD,GKQIJYDH,6,0.582501
6,AANGBULD,KJNSOAHR,3,0.400653
7,AANGBULD,LLHREMKS,5,0.499734
8,AANGBULD,NAQIHUKZ,2,0.525490
9,AANGBULD,OKMBFVKS,6,-0.483620


## Step 8: 결과 확인

In [10]:
import numpy as np
import pandas as pd
from sklearn.linear_model import LinearRegression
from sklearn.metrics import mean_absolute_error, mean_squared_error, r2_score

def predict_next_month_value(pairs, pivot, target_month='2025-08'):
    """
    공행성 쌍을 이용해 후행 품목의 다음 달 무역량 예측

    Parameters:
    - pairs: 공행성 쌍 데이터프레임
    - pivot: 품목×월 무역량 데이터
    - target_month: 예측할 목표 월 (기본: 2025-08)

    Returns:
    - predictions: 예측 결과 데이터프레임
    """

    predictions = []
    months = pivot.columns.to_list()

    for _, row in pairs.iterrows():
        leader = row['leading_item_id']
        follower = row['following_item_id']
        lag = row['best_lag']

        # 선행 품목의 시계열
        x_series = pivot.loc[leader].values.astype(float)

        # 후행 품목의 시계열
        y_series = pivot.loc[follower].values.astype(float)

        # 학습 데이터 준비 (lag 고려)
        X_train = []
        y_train = []

        for i in range(len(x_series) - lag):
            X_train.append([x_series[i]])  # 선행 품목의 t 시점
            y_train.append(y_series[i + lag])  # 후행 품목의 t+lag 시점

        X_train = np.array(X_train)
        y_train = np.array(y_train)

        # 선형 회귀 모델 학습
        model = LinearRegression()
        model.fit(X_train, y_train)

        # 예측: 가장 최근 선행 품목 값으로 다음 달 예측
        latest_leader_value = x_series[-lag]  # lag개월 전 값
        predicted_value = model.predict([[latest_leader_value]])[0]

        # 음수 방지
        predicted_value = max(0, predicted_value)

        predictions.append({
            'leading_item_id': leader,
            'following_item_id': follower,
            'best_lag': lag,
            'max_corr': row['max_corr'],
            'predicted_value': predicted_value,
            'model_coef': model.coef_[0],
            'model_intercept': model.intercept_
        })

    pred_df = pd.DataFrame(predictions)
    return pred_df

# 실행
predictions = predict_next_month_value(pairs, pivot, target_month='2025-08')
print(f"총 {len(predictions)}개 후행 품목의 2025년 8월 무역량 예측 완료")
print("\n예측 결과 샘플:")
predictions.head(20)

총 1371개 후행 품목의 2025년 8월 무역량 예측 완료

예측 결과 샘플:


,leading_item_id,following_item_id,best_lag,max_corr,predicted_value,model_coef,model_intercept
0,AANGBULD,APQGTRMF,5,-0.443984,2.888667e+05,-0.785181,2.888667e+05
1,AANGBULD,DEWLVASR,6,0.640221,1.948945e+05,0.960750,1.701023e+05
2,AANGBULD,DNMPSKTB,4,-0.410635,6.425620e+06,-5.811172,6.649007e+06
3,AANGBULD,EVBVXETX,6,0.436623,4.335038e+06,2.333127,4.274832e+06
4,AANGBULD,FTSVTTSR,3,0.531400,1.065339e+05,1.115567,1.065339e+05
5,AANGBULD,GKQIJYDH,6,0.582501,1.129536e+07,62.171664,9.691025e+06
6,AANGBULD,KJNSOAHR,3,0.400653,5.386476e+06,14.596738,5.386476e+06
7,AANGBULD,LLHREMKS,5,0.499734,3.314918e+04,0.307892,3.314918e+04
8,AANGBULD,NAQIHUKZ,2,0.525490,2.683782e+04,0.058487,1.029001e+03
9,AANGBULD,OKMBFVKS,6,-0.483620,2.360783e+06,-1.322346,2.394906e+06


In [12]:
# ============================================
# 제출 파일 생성 (test 데이터 없는 경우)
# ============================================

# Step 1: predictions에서 제출 형식으로 변환
submission = predictions[['leading_item_id', 'following_item_id', 'predicted_value']].copy()

# 컬럼명 변경
submission = submission.rename(columns={'predicted_value': 'value'})

# Step 2: 음수 방지
submission['value'] = submission['value'].clip(lower=0)

# Step 3: 같은 쌍이 중복되면 평균 (혹시 모를 중복 방지)
submission = submission.groupby(['leading_item_id', 'following_item_id'], as_index=False)['value'].mean()

# Step 4: CSV 저장
submission.to_csv('submission.csv', index=False)

print("="*60)
print("제출 파일 생성 완료!")
print("="*60)
print(f"총 제출 행 수: {len(submission)}")
print(f"평균 예측값: {submission['value'].mean():,.0f}")
print(f"최대 예측값: {submission['value'].max():,.0f}")
print(f"최소 예측값: {submission['value'].min():,.0f}")
print("="*60)

# 결과 미리보기
print("\n제출 파일 미리보기:")
print(submission.head(20))

print("\n✓ submission.csv 파일이 생성되었습니다!")

제출 파일 생성 완료!
총 제출 행 수: 1371
평균 예측값: 4,528,384
최대 예측값: 112,063,697
최소 예측값: 0

제출 파일 미리보기:
   leading_item_id following_item_id         value
0         AANGBULD          APQGTRMF  2.888667e+05
1         AANGBULD          DEWLVASR  1.948945e+05
2         AANGBULD          DNMPSKTB  6.425620e+06
3         AANGBULD          EVBVXETX  4.335038e+06
4         AANGBULD          FTSVTTSR  1.065339e+05
5         AANGBULD          GKQIJYDH  1.129536e+07
6         AANGBULD          KJNSOAHR  5.386476e+06
7         AANGBULD          LLHREMKS  3.314918e+04
8         AANGBULD          NAQIHUKZ  2.683782e+04
9         AANGBULD          OKMBFVKS  2.360783e+06
10        AANGBULD          UIFPPCLR  2.283199e+05
11        AANGBULD          VUAFAIYJ  8.630137e+05
12        AANGBULD          ZCELVYQU  8.418327e+04
13        AANGBULD          ZKENOUDA  5.198421e+05
14        AHMDUILJ          APQGTRMF  1.378256e+05
15        AHMDUILJ          ATLDMDBO  5.007958e+07
16        AHMDUILJ          BUZIIBYG  1.6974

In [15]:
import shutil

# submission.csv를 outputs 폴더로 복사
shutil.copy('submission.csv', '/content/drive/MyDrive/국민대_무역_공모전/submission.csv')

print("✓ 파일이 준비되었습니다!")

✓ 파일이 준비되었습니다!


In [ ]:
print("안녕")